# Ignore this bit

This bit of code was originally made when I was using csv files to emulate using a database just to try out the algorithm. The code below overrites the exisitng "site.db".

In [1]:
#import pandas as pd
#import sqlite3

# Read CSV files
#debt_item = pd.read_csv('falsedata.csv')
#user = pd.read_csv('falseusers.csv')

# Establish a connection to the database
#conn = sqlite3.connect('site.db')

# Write dataframes to SQLite tables
#debt_item.to_sql('debt_item', conn, if_exists='replace', index=False)
#user.to_sql('user', conn, if_exists='replace', index=False)

# Commit changes and close connection
#conn.commit()
#conn.close()

# Pre-Initialization; Read user table

This code loads the `user` table from our database and initializes it into a variable called `users`. 
This bit of code is not necessary for the algorithm itself, except for the print statement inside of it. 

Essentially it was made so that it can say "Alice" instead of "1" in the output. 

In [3]:
import pandas as pd
import sqlite3

# Import the debt_item table into a DataFrame named transactions
def import_debt_item(database_file):
    conn = sqlite3.connect(database_file)
    transactions = pd.read_sql_query("SELECT * FROM user", conn)
    conn.close()
    return transactions

# Example usage:
users = import_debt_item('site.db')

In [4]:
users.head()

,id,email,first_name,last_name,password_hash
0,0,alice@hs,Alice,Alpha,kdj;alkjdsa
1,1,bob@hs,Bob,Beta,akdjaslk;jda
2,2,ch@hs,Charlie,Omega,alldjaslk;j;


Every column in the debt_table in the database in this github is exactly as it was in the origianl.

# Hellgorithm

In the original code, the algoritm took in an adjacency matrix. The code here is pristine to the original found in: https://github.com/ATMackay/splitwise

The only change is the print statement which returns something useful. You can change the print statement if you want it to do something else. 
It's under: 

```python 
    def solveTransaction(self):
```

If you want to work on it further to directly take in the values from a database instead of the two-step process of converting to a matrix, be my guest. 

In [6]:
import heapq

# Comparator that will be used to make priority_queue
# containing pair of integers maxHeap Comparison is based
# on second entry in the pair which represents cash
# credit/debit
class AscCmp:
    def __call__(self, p1, p2):
        return p1[1] < p2[1]

# Comparator that will be used to make priority_queue
# containing pair of integers minHeap Comparison is based
# on second entry in the pair which represents cash
# credit/debit
class DscCmp:
    def __call__(self, p1, p2):
        return p1[1] > p2[1]

class Solution:
    def __init__(self):
        self.minQ = []
        self.maxQ = []

    # This function will fill in minQ and maxQ in such a
    # way that maxQ will have only positive value. minQ
    # will have only negative value amount is taken as
    # input. amount[i] => cash credit/debit to/from person
    # i amount[i] == 0 will be ignored as no credit/debit
    # is left.
    def constructMinMaxQ(self, amount):
        for i in range(len(amount)):
            if amount[i] == 0:
                continue
            if amount[i] > 0:
                heapq.heappush(self.maxQ, (i, amount[i]))
            else:
                heapq.heappush(self.minQ, (i, amount[i]))

    # This function will iterate over minQ and maxQ until
    # empty. It will fetch max credit and min debit. If sum
    # of both is not equal to zero, then push remaining
    # credit or debit back to the required queue. At the
    # end of the loop, print result
    def solveTransaction(self):
        while self.minQ and self.maxQ:
            maxCreditEntry = heapq.heappop(self.maxQ)
            maxDebitEntry = heapq.heappop(self.minQ)

            transaction_val = maxCreditEntry[1] + maxDebitEntry[1]

            debtor = maxDebitEntry[0]
            creditor = maxCreditEntry[0]
            owed_amount = 0

            if transaction_val == 0:
                owed_amount = maxCreditEntry[1]
            elif transaction_val < 0:
                owed_amount = maxCreditEntry[1]
                heapq.heappush(self.minQ, (maxDebitEntry[0], transaction_val))
            else:
                owed_amount = -maxDebitEntry[1]
                heapq.heappush(self.maxQ, (maxCreditEntry[0], transaction_val))

            print(f"{users[users['id'] == debtor]['first_name'].values[0]} pays {owed_amount} euros to {users[users['id'] == creditor]['first_name'].values[0]}")
            
            

    def minCashFlow(self, graph):
        n = len(graph)

        # Calculate the cash to be credited/debited to/from
        # each person and store in vector amount
        amount = [0] * n
        for i in range(n):
            for j in range(n):
                diff = graph[j][i] - graph[i][j]
                amount[i] += diff

        # Fill in both queues minQ and maxQ using amount
        # vector
        self.constructMinMaxQ(amount)

        # Solve the transaction using minQ, maxQ and amount
        # vector
        self.solveTransaction()



# Read Database (debts table) and Convert to Matrix
Made by chat-GPT. Feel free to simplify it if you can find a way.
* Takes in a database file
* Retrieves appropriate information (i.e. user_ids of the giver and receiver, and the amount)
* Constructs and returns the adjacency matrix for the algorithm above


In [7]:
import sqlite3

def read_db_to_adjacency_matrix(database_file):
    persons = set()  # Store unique persons

    # Connect to the SQLite database
    conn = sqlite3.connect(database_file)
    cursor = conn.cursor()

    # Query the database to determine unique persons
    cursor.execute("SELECT giver, receiver FROM debt_item")
    rows = cursor.fetchall()
    for row in rows:
        giver = row[0]
        receiver = row[1]
        persons.add(giver)
        persons.add(receiver)

    persons = sorted(persons)  # Sort persons alphabetically

    n = len(persons)
    adjacency_matrix = [[0] * n for _ in range(n)]

    # Populate the adjacency matrix with transaction amounts
    cursor.execute("SELECT giver, receiver, amount FROM debt_item")
    rows = cursor.fetchall()
    for row in rows:
        giver = row[0]
        receiver = row[1]
        amount = row[2]

        i = persons.index(giver)
        j = persons.index(receiver)
        adjacency_matrix[i][j] += amount

    # Close the database connection
    conn.close()

    return adjacency_matrix




## Just checking if the function above works properly. 
* The database is called through above function (to convert table to matrix) and intialized into a variable.

In [8]:
# Example usage:
graph1 = read_db_to_adjacency_matrix('site.db')
for row in graph1:
    print(row)



[0, 5, 1]
[0, 0, 5]
[10, 0, 0]


## Run the algorithm on the initialized matrix.

In [9]:
S = Solution()
S.minCashFlow(graph1)

Charlie  pays 4 euros to Alice 


---
<font color='red'> TO MAKE SURE THIS CODE WORKS PROPERLY IN THE SHARED REPO: From the existing db, please delete "debtor_1, debtor_2, debtor_3" accordingly. Make sure the db matches the columns that exist in falseusers and falsedata. This is because the function to read database and convert uses indices. </font>. 
---

# Process

1) Have database with tables
2) Pass the database `read_db_to_adjacency_matrix()`
3) feed the matrix from step 2 into the algorithm's function

Database  $\rightarrow$ `read_db_to_adjacency_matrix()`  $\rightarrow$ Assign **matrix** to an object/variable 

Assign `Solution` class to an object/variable $\rightarrow$  Run the algorithm with {solution object}`.minCashFlow`({matrix})